![](https://fully-connected-graph.github.io/GMLF-2023/assets/banner.png)

# Introduction to Recommender Systems

This is practical session is part of the first lecture of the [Machine Learning Fortnight](https://mlfortnight.svcover.nl/) organized by the [Fully Connected Graph](https://svcover.nl/committees?commissie=programming_committee).

Alongside the lectures there is a competition on Kaggle, register for it to copmete with others and test your knowledge.

## Movie Recommendation Practical Session

In this practical session you will get hands-on practice in building recommendation systems. The dataset we are exploring is the MovieLens 100K movie ratings. It is made up of user-item pairs as well as relevant data.

In [ ]:
# Download the necessary libraries
!pip install faiss-cpu

In [ ]:
# import the nessary libraries

import pandas as pd
import numpy as np
import os
import tensorflow as tf
import faiss

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error as mae

Let's download the dataset

In [ ]:
%%capture

# download the dataset if not already downloaded
if not os.path.exists('ml-100k'):
    !wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
    !unzip ml-100k.zip
    !rm ml-100k.zip

This data set consists of:
	
* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
    * Simple demographic info for the users (age, gender, occupation, zip)

Let's see the structure of the dataset, there are 3 important files:
- u.data: contains the edges/reviews from each user to a movie.
- u.user: details of a user.
- u.item: details of a movie.

Aside from these are auxiliary files which add context to the dataset.
It's a good idea to check the README file to understand the dataset better.

## Data Exploration

Let's examine each of the important files and process them into a more usable format. We will first look at the data file which contains the reviews of each user for a movie


In [ ]:
# Load the rating data into a dataframe

ratings = pd.read_csv(
    'ml-100k/u1.base', sep='\t', 
    names=['user_id', 'movie_id', 'rating', 'timestamp']
)
ratings.head()

In [ ]:
# Load genres data into a dataframe

genre_data = pd.read_csv(
    'ml-100k/u.genre', sep='|', 
    names=['genre', 'genre_id']
)
genre_data.head()

In [ ]:
# Load movie data into a dataframe

movie_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + genre_data['genre'].tolist()

movie_data = pd.read_csv(
    'ml-100k/u.item', sep='|', 
    names=movie_cols,
    encoding='latin-1'
)

# Make movie_id the index
movie_data.set_index('movie_id', inplace=True)

movie_data.head()

In [ ]:
# Load user data into a dataframe

user_cols = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

user_data = pd.read_csv(
    'ml-100k/u.user', sep='|', 
    names=user_cols,
    encoding='latin-1'
)

user_data.head()

## Cleaning the Data

Let's drop the irrelevant data, such as

From `user_data` we will drop the following columns:
- `zip code`
    - Geographical location might be correlated with user preferences, but it is complex to process

From `ratings`, we will drop the following columns
- `timestamps` for user-movie pair reviews (u.data)
    - time can be used as a feature, because user preferences change over time, but we will not use it in this practical session.m `movie_data` we will drop the following columns:
- `movie title`
    - we will use the movie id to identify movies
- `release date`
- `video release date`
- `IMDb URL`

<details>
    <summary>Answer</summary>

```python
# Drop irrelevant columns
user_data.drop('zip code', axis=1, inplace=True)
ratings.drop('timestamp', axis=1, inplace=True)
movie_data.drop(['movie title', 'release date', 'video release date', 'IMDb URL'], axis=1, inplace=True)
```

</details>
    

In [ ]:
# TODO: Fill in the columns to remove

ratings.drop(columns=[
    # YOUR CODE HERE
], inplace=True)

user_data.drop(columns=[
    # YOUR CODE HERE
], inplace=True)

movie_data.drop(columns=[
    # YOUR CODE HERE
], inplace=True)

## One Hot Encoding of Categorical Data

The user table contains information about their occupation and gender which is represented in text. 
It is necessary to convert this column into a more usable format

<details>
    <summary>Answer</summary>

```python
encoded_user_data = pd.get_dummies(encoded_user_data, columns=[
    'gender', 'occupation'
], drop_first=True, dtype='int')
```

</details>

In [ ]:
encoded_user_data = user_data.copy()

# OHE the occupation and gender columns
encoded_user_data = pd.get_dummies(encoded_user_data, columns=[
    # YOUR CODE HERE
], drop_first=True, dtype='int')
encoded_user_data.head()

## Normalization of Numerical Data

The age of users and the rating of movies are numerical data, but they are not on the same scale. We will also convert the ratings column into 0 and 1 to simplify training.

You can use StandardScaler from sklearn to normalize the data.

<details>
    <summary>Answer</summary>

```python
age_scaler = StandardScaler()
encoded_user_data['age'] = age_scaler.fit_transform(encoded_user_data[['age']])

ratings['rating'] = ratings['rating'] / 5.0
```

</details>

In [ ]:
# TODO: Normalize user age and rating

# YOUR CODE HERE

# Divide rating by 5
# YOUR CODE HERE

## Loading Test Data

Our training data is obtained from u1.base which contains 80% of the data. We will use `u1.test` to test our model which contains the remaining 20%.

<details>
    <summary>Answer</summary>

```python

test_ratings.drop(columns=['timestamp'], inplace=True)

test_ratings['rating'] = test_ratings['rating'] / 5.0

```

</details>

In [ ]:
# Apply the same process (of normalization) to the test data

test_ratings = pd.read_csv(
    'ml-100k/u1.test', sep='\t', 
    names=['user_id', 'movie_id', 'rating', 'timestamp']
)

# YOUR CODE HERE

test_ratings.head()

## Content Based Filtering

In content based filtering, we will use the features of the items to recommend similar items to the user.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*Lr6qL0YjY_WqVK5u-AYHAQ.png" width="360" height="240" />

There are different metrics that we can use to compute similarity between items. In order for these metrics to work, we can think of the tables as multidimensional vectors.

### Euclidian Distance

This considers the distance between vector heads. Accounts for magnitude and direction.

$$
\begin{align}
\sqrt{\sum_{i=1}^n (x_i-y_i)^2}     
\end{align}
$$

### Dot Product

Computes the products between components. Accounts for magnitude and direction.

$$
\begin{align}
\sum_{i=1}^n (x_i.y_i)  
\end{align}
$$

### Cosine Similarity

This metric considers the angle between the vectors. Accounts for direction only.

$$
\begin{align}
cos(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}
\end{align}
$$

A useful library is FAISS (Facebook AI Similarity Search), which is a library for efficient similarity search and clustering of dense vectors. It contains implementations of the above metrics and more.

## Implement K Nearest Neighbors

The kNN algorithm can be used to find the k most similar items to a given item. You can read the documentation of FAISS [here](https://faiss.ai/cpp_api/struct/structfaiss_1_1IndexFlatL2.html)

<details>
    <summary>Answer</summary>

```python

# Create index with idmap
database = faiss.IndexIDMap(faiss.IndexFlatL2(movie_shape))
database.add_with_ids(content_movie_data.values, content_movie_data.index.values)

```
</details>

In [ ]:
movie_titles = movie_data[['title']]
content_movie_data = movie_data.drop(columns=['title'])

# Get shape of embeddings
movie_shape = content_movie_data.shape[1]

# TODO: Create a Database for the movies
## YOUR CODE HERE
database = ...

In [ ]:
# Code to debug the predictions
def display_movie(movie_id, item_data):

    # Get the movie title
    movie_title = movie_titles.loc[movie_id]['title']
    print("Movie title:", movie_title)

def display_predictions(item_data, movie_id, closest_movie_ids):

    print("Original movie")
    display_movie(movie_id, item_data)

    for index in range(len(closest_movie_ids)):
        print("Recommendation", index + 1)
        display_movie(closest_movie_ids[index], item_data)

<details>

<summary>Answer</summary>

```python
_, closest_movie_ids = self.database.search(random_movie_dimensions.reshape(1, -1), k_neighbours)
```

</details>

In [ ]:
class ContentModel:

    def __init__(self, train_data, item_data, database):
        self.known_knowledge = train_data
        self.item_data = item_data
        self.database = database


    # Select a random user, get a random movie they rated, and return a movie that is similar to that movie
    def get_recommendation(self, user_id, rating, verbose=False, k_neighbours=3): 

        # If rating > 0.5, set ascending to False, else set it to True
        order = rating >= 0.5

        # Get the best or worst rated movie by the user
        movie_id = self.known_knowledge.loc[(self.known_knowledge['user_id'] == user_id)].sort_values(by='rating', ascending=order).iloc[0]['movie_id'] 
        
        # Get the dimensions of the movie
        random_movie_dimensions = self.item_data.loc[movie_id].values

        # TODO: Get the closest movie using faiss

        # Get the closest movie using faiss, 
        # first var is distance, second is index
        closest_movie_ids = ## YOURS CODE HERE
        

        if verbose:
            display_predictions(self.item_data, movie_id, closest_movie_ids[0])

        # Calculate the mean of the ratings of the closest movies
        mean = self.known_knowledge[self.known_knowledge['movie_id'].isin(closest_movie_ids[0])]['rating'].mean()

        return mean

In [ ]:
content_model = ContentModel(ratings, content_movie_data, database)

# Get a random row from the training data
random_row = test_ratings.sample(1)

content_model.get_recommendation(random_row['user_id'].values[0], random_row['rating'].values[0], verbose=True)

The score you get is the average rating of the k most similar items.

In [ ]:
# evaluate the model on test data

def evaluate_model(model, test_data):
    # YOUR CODE HERE
    predictions = []
    for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
        predictions.append(model.get_recommendation(row['user_id'], row['rating']))
    
    return np.array(predictions)

predictions = evaluate_model(content_model, test_ratings)

## Evaluate the model
Now that we have the predictions, we can use a metric to evaluate the model. Recall that our task is regression based

<details>
    <summary>Answer</summary>

```python
print("MAE:", mae(test_ratings['rating'], predictions))
```

</details>

In [ ]:
## TODO: Calculate the Error

## YOUR CODE HERE

## Collaborative Filtering

Now that the data preprocessing part has been taken care of, let's get to the more exciting part, the algorithm. The algorithm that we'll introduce today is Matrix Factorization.

Recall that we had a user-item matrix, $R$, where nonzero elements of the matrix are ratings that a user has given an item. What Matrix Factorization does is it decomposes a large matrix into products of matrices, namely, $R = U \times V$. See the picture below taken from a quick google search for a better understanding:

![R = U x V](https://ethen8181.github.io/machine-learning/recsys/img/matrix_factorization.png)

Matrix factorization assumes that:
- Each user can be described by $d$ features. For example, feature 1 might be a referring to how much each user likes disney movies.
- Each item, movie in this case, can be described by an analogous set of $d$ features. To correspond to the above example, feature 1 for the movie might be a number that says how close the movie is to a disney movie.
After learning the two smaller matrices, all we have to do is to perform a matrix multiplication of the two matrices and the end result will be a our approximation for the rating the user would give that item (movie).

The cool thing about this is that, we do not know what these features are nor do we have to determine them beforehand, which is why these features are often refer to as latent features. Next, we also don't know how many latent features are optimal for the task at hand. Thus, we can use random search or grid search or other fancy techniques to perform hyperparameter tuning and determine the best number for $d$.

Given all those information, the next question is: how do we learn the user matrix, $U$, and item matrix, $V$? Well, like a lot of machine learning algorithm, by minimizing a loss function.

### Loss Function

We start by denoting our $d$ feature user into math by letting a user $u$ take the form of a $1 \times d$-dimensional vector $\textbf{x}_{u}$. These for often times referred to as latent vectors or low-dimensional embeddings. Similarly, an item *i* can be represented by a $1 \times d$-dimensional vector $\textbf{y}_{i}$. And the rating that we predict user $u$ will give for item $i$ is just the dot product of the two vectors


$$
\begin{align}
\hat r_{ui} &= \textbf{x}_{u} \textbf{y}_{i}^{T} = \sum\limits_{d} x_{ud}y_{di}
\end{align}
$$

Where $\hat r_{ui}$ represents our prediction for the true rating $r_{ui}$. Next, we will choose a objective function to minimize the square of the difference between all ratings in our dataset ($S$) and our predictions. This produces a objective function of the form:

$$
\begin{align}
L &= \sum\limits_{u,i \in S}( r_{ui} - \textbf{x}_{u} \textbf{y}_{i}^{T} )^{2} + \lambda \big( \sum\limits_{u} \left\Vert \textbf{x}_{u} \right\Vert^{2} + \sum\limits_{i} \left\Vert \textbf{y}_{i} \right\Vert^{2} \big)
\end{align}
$$

Note that we've added on two $L_{2}$ regularization terms, with $\lambda$ controlling the strength at the end to prevent overfitting of the user and item vectors. $\lambda$, is another hyperparameter that we'll have to search for to determine the best value. The concept of regularization can be a topic of itself, and if you're confused by this, consider checking out [this documentation](http://nbviewer.jupyter.org/github/ethen8181/machine-learning/blob/master/regularization/regularization.ipynb) that covers it a bit more.

### Optimization Algorithm

Now that we formalize our objective function, we'll introduce the **Alternating Least Squares with Weighted Regularization (ALS-WR)** method for optimizing it. The way it works is we start by treating one set of latent vectors as constant. For this example, we'll pick the item vectors, $\textbf{y}_{i}$. We then take the derivative of the loss function with respect to the other set of vectors, the user vectors, $\textbf{x}_{u}$ and solve for the non-constant vectors (the user vectors).

$$
\begin{align}
\frac{\partial L}{\partial \textbf{x}_{u}} 
&= - 2 \sum\limits_{i}(r_{ui} - \textbf{x}_{u} \textbf{y}_{i}^{T} ) \textbf{y}_{i} + 2 \lambda \textbf{x}_{u} = 0 \\
&= -(\textbf{r}_{u} - \textbf{x}_{u} Y^{T} )Y + \lambda \textbf{x}_{u} = 0 \\
&= \textbf{x}_{u} (Y^{T}Y + \lambda I) = \textbf{r}_{u}Y \\
&= \textbf{x}_{u} = \textbf{r}_{u}Y (Y^{T}Y + \lambda I)^{-1}
\end{align}
$$

To clarify it a bit, let us assume that we have $m$ users and $n$ items, so our ratings matrix is $m \times n$.

- The row vector $\textbf{r}_{u}$ represents users *u*'s row from the ratings matrix with all the ratings for all the items (so it has dimension $1 \times n$)
-  We introduce the symbol $Y$, with dimensions $n \times d$, to represent all item row vectors vertically stacked on each other
- Lastly, $I$ is the identity matrix which has dimension $d \times d$ to ensure the matrix multiplication's dimensionality will be correct when we add the $\lambda$

Now comes the alternating part: With these newly updated user vectors in hand, in the next round, we hold them as constant, and take the derivative of the loss function with respect to the previously constant vectors (the item vectors). As the derivation for the item vectors is quite similar, we will simply list out the end formula:

$$
\begin{align}
\frac{\partial L}{\partial \textbf{y}_{i}}
&= \textbf{y}_{i} = \textbf{r}_{i}X (X^{T}X + \lambda I)^{-1}
\end{align}
$$

Then we alternate back and forth and carry out this two-step process until convergence. The reason we alternate is, optimizing user latent vectors, $U$, and item latent vectors $V$ simultaneously is hard to solve. If we fix $U$ or $V$ and tackle one problem at a time, we potentially turn it into a easier sub-problem. Just to summarize it, ALS works by:

1. Initialize the user latent vectors, $U$, and item latent vectors $V$ with randomly
2. Fix $U$ and solve for $V$
3. Fix $V$ and solve for $U$
4. Repeat step 2 and 3 until convergence

Now that we have our equations, let's program this thing up! We'll set the model to use 20 factors and a regularization value of 0.01 (chosen at random) and train it for 100 iterations and compute the mean square error on the train and test set to assess algorithm convergence.

### Implementation

Let's first make the user and movies matricies and initialize them with random values.

Use the [`np.random.normal()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html) function to generate random values for the user and movie matrices.

Use the scale parameter to set the standard deviation of the distribution to `1 / n_users` and `1 / n_movies` respectively.

<details>
<summary>Answer</summary>

<pre>
<code>
user_matrix = np.random.normal(
    scale=1./n_users, 
    size=(n_users, embed_dim)
)
item_matrix = np.random.normal(
    scale=1./n_items, 
    size=(n_items, embed_dim)
)
</code>
</pre>

</details>

In [ ]:
embed_dim = 300

n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

user_matrix = np.random.normal( # YOUR CODE HERE
    scale=1./n_users, 
    size=(n_users, embed_dim)
)
item_matrix = np.random.normal( # YOUR CODE HERE
    scale=1./n_items, 
    size=(n_items, embed_dim)
)

Let's implement a helper predict function. It calculates the dot product of the user and movie matrices.

<details>
<summary>Answer</summary>

<pre>
<code>
def predict(user_matrix, item_matrix):
    """predict ratings for every user and item"""
    pred = user_matrix.dot(item_matrix.T) # TODO: YOUR CODE HERE
    return pred
</code>
</pre>

</details>

In [ ]:
def predict(user_matrix, item_matrix):
    """predict ratings for every user and item"""
    pred = user_matrix.dot(item_matrix.T) # TODO: YOUR CODE HERE
    return pred

and a helper function to compute the mean square error

In [ ]:
from sklearn.metrics import mean_squared_error

def compute_mse(y_pred, y_true):
    """ignore zero terms prior to comparing the mse"""
    mask = np.nonzero(y_true)
    mse = mean_squared_error(y_true[mask], y_pred[mask])
    return mse

Now let's implement one step of the ALS algorithm. Follow the steps from the formula above.

<details>
<summary>Answer</summary>

<pre>
<code>
def als_step(ratings, solve_vecs, fixed_vecs, regulazation_lambda = 0.01):
    """
    when updating the user matrix,
    the item matrix is the fixed vector and vice versa
    """
    A = fixed_vecs.T.dot(fixed_vecs) + np.eye(embed_dim) * regulazation_lambda
    b = ratings.dot(fixed_vecs)
    A_inv = np.linalg.inv(A)
    solve_vecs = b.dot(A_inv)
    return solve_vecs
</code>
</pre>

</details>

In [ ]:
def als_step(ratings, solve_vecs, fixed_vecs, regulazation_lambda = 0.01):
    """
    when updating the user matrix,
    the item matrix is the fixed vector and vice versa
    """

    # YOUR CODE HERE

    A = fixed_vecs.T.dot(fixed_vecs) + np.eye(embed_dim) * regulazation_lambda
    b = ratings.dot(fixed_vecs)
    A_inv = np.linalg.inv(A)
    solve_vecs = b.dot(A_inv)
    return solve_vecs

Let's make the training data into a sparse matrix.

In [ ]:
train = ratings.copy()
train = train.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

test = test_ratings.copy()
test = test.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

In [ ]:
"""
pass in training and testing at the same time to record
model convergence, assuming both dataset is in the form
of User x Item matrix with cells as ratings
"""

n_iters = 100
LAMBDA = 0.01

# record the training and testing mse for every iteration
# to show convergence later (usually, not worth it for production)
train_mse_record = []
test_mse_record  = []
for _ in tqdm(range(n_iters)):
    user_matrix = als_step(train.values, user_matrix, item_matrix, regulazation_lambda=LAMBDA)
    item_matrix = als_step(train.values.T, item_matrix, user_matrix, regulazation_lambda=LAMBDA)
    predictions = predict(user_matrix, item_matrix)
    train_mse = compute_mse(train.values, predictions)

    test_user_matrix = user_matrix[test.index.values - 1]
    test_item_matrix = item_matrix[test.columns.values - 1]
    test_predictions = predict(test_user_matrix, test_item_matrix)
    test_mse = compute_mse(test.values, test_predictions)
    
    test_mse_record.append(test_mse)
    train_mse_record.append(train_mse)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(n_iters), train_mse_record, label='Train', color='red')
plt.plot(range(n_iters), test_mse_record, label='Test', color='blue')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('MSE')
plt.show()

Notice that ALS converges after a few sweeps, which is one of the main reason for its popularity. Fast, thus scalable to bigger dataset.

Possibly your model has overfitted, to prevent this, you can play with the regularization parameter, embedding dimension, and number of iterations.

<details>
<summary>Answer to what lambda regularization parameter to choose</summary>

From our experiments, we found that a value of from 20 to 50 work well.

</details>

Let's evaluate it with MAE

<details>
<summary>Answer</summary>

<pre>
<code>
print("MAE:", mae(test.values, test_predictions))
</code>
</pre>

</details>

In [ ]:
# TODO: Calculate the Error for the ALS model on the test data

# YOUR CODE HERE